In [50]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import os
from keras.layers import Dense, LSTM, Embedding
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
pwd = os.getcwd()
pwd

'c:\\Users\\33645\\Desktop\\projects\\kaggle\\FakeNews'

In [51]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [52]:
train = pd.read_csv('train.csv', usecols=['title', 'author', 'text', 'label'])
test = pd.read_csv('test.csv', usecols=['title', 'author', 'text'])
train.head()

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [53]:
# df=pd.concat([train,test],axis='rows')
df['comb']=df['author']+" "+df['title']     # Combined features 
test['comb']=df['author']+" "+df['title']

In [54]:
#Number of Null values
df.isna().sum().sum()

5069

In [55]:
#replace na values
df.fillna('unavailable',inplace=True)
df.reset_index(inplace=True)
test.fillna('unavailable',inplace=True)
test.reset_index(inplace=True)

In [56]:
#Getting indedependent variable and independent variables
X = df.drop('label',axis=1)
y = df['label']


In [57]:
#Make a copy in case I want to go back
X_edited = X.copy()
len(X_edited)

20800

# preprocessing Text 

In [58]:
#init a list to contain preprocessed text
corpus = []
st = PorterStemmer()
# lemmatizer = WordNetLemmatizer()

for i, title in enumerate(X_edited['comb']):
    title_edited = re.sub('[^a-zA-Z]',' ', title)
    title_edited_list = title_edited.lower().split()
    title_words_stem = [st.stem(word) for word in title_edited_list if word not in stopwords.words('english')]
    corpus.append(' '.join(title_words_stem))

len(corpus)


20800

In [59]:
#init a list to contain preprocessed text
X_test = []
st = PorterStemmer()
# lemmatizer = WordNetLemmatizer()

for i, title in enumerate(test['comb']):
    title_edited = re.sub('[^a-zA-Z]',' ', title)
    title_edited_list = title_edited.lower().split()
    title_words_stem = [st.stem(word) for word in title_edited_list if word not in stopwords.words('english')]
    X_test.append(' '.join(title_words_stem))

len(X_test)

5200

In [60]:
#example of preprocessed title
corpus[0]

'darrel lucu hous dem aid even see comey letter jason chaffetz tweet'

# Word Embeddings

In [61]:
#First step: One-hot Encode
voc_size = 2000
one_hot_repr = [one_hot(title, voc_size) for title in corpus]
one_hot_X_test = [one_hot(title, voc_size) for title in X_test]
#One example of one_hot_rep: we replace each word in the title by
#its index in the vocabulary
one_hot_repr[0]

[160, 627, 660, 262, 1206, 516, 1864, 393, 1093, 1312, 506, 704]

In [62]:
max([len(lst) for lst in one_hot_repr])

51

In [63]:
sentence_length=60
padded_docs=pad_sequences(one_hot_repr,padding='pre',maxlen=sentence_length)
padded_docs_test = pad_sequences(one_hot_X_test,padding='pre',maxlen=sentence_length)
padded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,  160,  627,  660,  262, 1206,  516, 1864,
        393, 1093, 1312,  506,  704])

## Creating model

In [64]:
#embedding_vector_features
output_dim=50
input_dim = voc_size
model=Sequential()
model.add(Embedding(input_dim,output_dim,input_length=sentence_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 60, 50)            100000    
                                                                 
 lstm_5 (LSTM)               (None, 100)               60400     
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 160,501
Trainable params: 160,501
Non-trainable params: 0
_________________________________________________________________
None


In [65]:
y.unique()

array([1, 0], dtype=int64)

In [66]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=4)
X_edited_bl, y_bl = sm.fit_resample(padded_docs, y)
X_train, X_val, y_train, y_val = train_test_split(X_edited_bl, y_bl, test_size=0.3, random_state=4)


In [71]:
model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=20,batch_size=64)

Epoch 1/20
228/228 [==============================] - 11s 47ms/step - loss: 0.0066 - accuracy: 0.9989 - val_loss: 0.0719 - val_accuracy: 0.9861
Epoch 2/20
228/228 [==============================] - 11s 47ms/step - loss: 0.0062 - accuracy: 0.9990 - val_loss: 0.0733 - val_accuracy: 0.9872
Epoch 3/20
228/228 [==============================] - 11s 47ms/step - loss: 0.0061 - accuracy: 0.9990 - val_loss: 0.0764 - val_accuracy: 0.9867
Epoch 4/20
228/228 [==============================] - 11s 47ms/step - loss: 0.0059 - accuracy: 0.9990 - val_loss: 0.0717 - val_accuracy: 0.9877
Epoch 5/20
228/228 [==============================] - 11s 47ms/step - loss: 0.0059 - accuracy: 0.9990 - val_loss: 0.0726 - val_accuracy: 0.9869
Epoch 6/20
228/228 [==============================] - 11s 47ms/step - loss: 0.0058 - accuracy: 0.9990 - val_loss: 0.0776 - val_accuracy: 0.9874
Epoch 7/20
228/228 [==============================] - 11s 47ms/step - loss: 0.0060 - accuracy: 0.9990 - val_loss: 0.0728 - val_accuracy:

In [69]:
import tensorflow as tf
sgd=tf.keras.optimizers.SGD(learning_rate=0.01, decay=1e-6,momentum=0.9, nesterov=True)
rms = tf.keras.optimizers.RMSprop()
nadam=tf.keras.optimizers.Nadam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam"
)

In [91]:
from keras.layers import Dropout
#embedding_vector_features
output_dim=50
input_dim = voc_size
model2=Sequential()
model2.add(Embedding(input_dim,output_dim,input_length=sentence_length))
model2.add(LSTM(100))
model2.add(Dropout(0.1))
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model2.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 60, 50)            100000    
                                                                 
 lstm_7 (LSTM)               (None, 100)               60400     
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                 101       
                                                                 
Total params: 160,501
Trainable params: 160,501
Non-trainable params: 0
_________________________________________________________________
None


In [93]:
model2.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=10,batch_size=64)

Epoch 1/10
228/228 [==============================] - 11s 47ms/step - loss: 0.0076 - accuracy: 0.9987 - val_loss: 0.0730 - val_accuracy: 0.9832
Epoch 2/10
228/228 [==============================] - 11s 47ms/step - loss: 0.0064 - accuracy: 0.9990 - val_loss: 0.0779 - val_accuracy: 0.9840
Epoch 3/10
228/228 [==============================] - 11s 47ms/step - loss: 0.0060 - accuracy: 0.9990 - val_loss: 0.0805 - val_accuracy: 0.9840
Epoch 4/10
228/228 [==============================] - 11s 47ms/step - loss: 0.0061 - accuracy: 0.9990 - val_loss: 0.0842 - val_accuracy: 0.9845
Epoch 5/10
228/228 [==============================] - 11s 48ms/step - loss: 0.0060 - accuracy: 0.9990 - val_loss: 0.0877 - val_accuracy: 0.9838
Epoch 6/10
228/228 [==============================] - 11s 46ms/step - loss: 0.0057 - accuracy: 0.9990 - val_loss: 0.0937 - val_accuracy: 0.9834
Epoch 7/10
228/228 [==============================] - 11s 46ms/step - loss: 0.0059 - accuracy: 0.9990 - val_loss: 0.0930 - val_accuracy:

## CHOOSING THE THRESHOLD

In [86]:
def cutoff(fpr,tpr,thresholds):
  difference = tpr-fpr
  sorted_diff_thresh_inc = sorted(zip(difference,thresholds))
  #return thresh giving the biggest diff
  return sorted_diff_thresh_inc[-1][1]

In [82]:
y_pred_train.ravel()

array([4.6331223e-09, 2.2366070e-08, 6.0099425e-11, ..., 1.0864171e-05,
       9.9994206e-01, 9.9998844e-01], dtype=float32)

In [87]:
from sklearn.metrics import precision_recall_curve, roc_curve
import numpy as np
y_pred_valid = model.predict(X_val)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(padded_docs_test)
precisions, recalls, thresholds = precision_recall_curve(y_val, y_pred_valid)
fpr, tpr, threshold = roc_curve(y_val, y_pred_valid)
thresh = cutoff(fpr, tpr, threshold)
pred_train = pd.DataFrame()
pred_train["label"] = y_pred_train.ravel()
pred_train["label"] = np.where(pred_train["label"] > float(thresh), 1, 0)

pred_valid = pd.DataFrame()
pred_valid["label"] = y_pred_valid.ravel()
pred_valid["label"] = np.where(pred_valid["label"] > float(thresh), 1, 0)

pred_test = pd.DataFrame()
pred_test['label'] = y_pred_test.ravel()
pred_test['label'] = np.where(pred_test['label'] > float(thresh), 1, 0)


In [90]:
from sklearn.metrics import f1_score
print(f1_score(y_train, pred_train["label"],average='macro'))
f1_score(y_val, pred_valid["label"],average='macro')

0.9990396071377126


0.9883142971974825